In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

# --- Numba Imports ---
from numba import jit, njit, prange

# ====================================================================
# --- SECTION 1: POTENTIAL & FORCE (Numba JIT) ---
# ====================================================================

@njit
def _E_p_prof(x, E0, x0, sigma0, E1, x1, sigma1):
    """
    Professor's ground truth potential: E_p = -E0*Gauss(x0,s0) - E1*Gauss(x1,s1)
    The (Norm_1D) is a Gaussian function:
    exp( -(x - center)**2 / (2 * sigma**2) )
    """
    well0 = -E0 * np.exp(-(x - x0)**2 / (2 * sigma0**2))
    well1 = -E1 * np.exp(-(x - x1)**2 / (2 * sigma1**2))
    return well0 + well1

@njit
def _F_p_prof(x, E0, x0, sigma0, E1, x1, sigma1):
    """
    Force from the professor's potential: F = -d(E_p)/dx
    d/dx[-E*exp(-(x-c)^2/(2s^2))] = E * (x-c)/s^2 * exp(...)
    Force = - (dE_well0/dx + dE_well1/dx)
    """
    f_well0 = E0 * np.exp(-(x - x0)**2 / (2 * sigma0**2)) * (x - x0) / sigma0**2
    f_well1 = E1 * np.exp(-(x - x1)**2 / (2 * sigma1**2)) * (x - x1) / sigma1**2
    return -(f_well0 + f_well1)


# ====================================================================
# --- SECTION 2: METADYNAMICS BIAS FUNCTIONS (Numba JIT) ---
# ====================================================================

@njit
def _V_bias(x, bias_centers, H_mtd, w_mtd):
    """Total bias potential V_bias(x) from all dropped Gaussians"""
    v_b = 0.0
    for i in prange(len(bias_centers)):
        x_c = bias_centers[i]
        v_b += H_mtd * np.exp(-(x - x_c)**2 / (2 * w_mtd**2))
    return v_b

@njit
def _F_bias(x, bias_centers, H_mtd, w_mtd):
    """Force from bias potential: F_bias = -d(V_bias)/dx"""
    f_b = 0.0
    for i in prange(len(bias_centers)):
        x_c = bias_centers[i]
        # d/dx[H*exp(...)] = H * exp(...) * [-(2(x-c))/(2s^2)]
        f_b += H_mtd * np.exp(-(x - x_c)**2 / (2 * w_mtd**2)) * \
               (-(x - x_c) / w_mtd**2)
    return -f_b # F = -dV/dx


# ====================================================================
# --- SECTION 3: MAIN SIMULATION LOOP (Numba JIT) ---
# ====================================================================

@jit(nopython=True) 
def run_simulation_loop(
    total_steps, dt, x_min, x_max,
    x_initial, v_initial, m, 
    gamma_eq, equilibration_time,
    gamma_mtd, temperature_mtd,
    tau_G_steps, H_mtd, w_mtd,
    potential_params, 
    print_interval_steps, seed
):
    """
    This is the core simulation loop, optimized for Numba.
    Uses Velocity-Verlet integrator.
    """
    
    # --- Unpack 6 parameters ---
    E0, x0, s0, E1, x1, s1 = potential_params

    # History arrays
    t_history = np.zeros(total_steps)
    x_history = np.zeros(total_steps)
    v_history = np.zeros(total_steps)
    bias_energy_history = np.zeros(total_steps)
    
    max_gaussians = total_steps // tau_G_steps + 1
    bias_centers = np.zeros(max_gaussians, dtype=np.float64)
    num_bias_centers = 0
    total_bias_added = 0.0
    
    x = x_initial
    v = v_initial
    steps_since_last_drop = 0
    np.random.seed(seed)

    # Calculate initial force (used only to start the Verlet loop)
    F_p = _F_p_prof(x, E0, x0, s0, E1, x1, s1)
    
    # Initial Force Calculation is split for Numba compatibility
    if 0 < equilibration_time:
        F_total = F_p - gamma_eq * v
    else:
        # F_p - gamma*v + R + F_bias(initially 0)
        random_force = np.random.normal(0, np.sqrt(2 * gamma_mtd * temperature_mtd / dt)) 
        F_total = F_p - gamma_mtd * v + random_force + _F_bias(x, bias_centers[:num_bias_centers], H_mtd, w_mtd)
    
    a = F_total / m
    
    # Main Velocity-Verlet Loop
    for i in range(total_steps):
        
        v_half = v + a * (dt / 2.0)
        x_new = x + v_half * dt
        
        # Reflective boundary conditions
        if x_new < x_min:
            x_new = x_min + (x_min - x_new); v_half = -v_half
        elif x_new > x_max:
            x_new = x_max - (x_new - x_max); v_half = -v_half
        
        x = x_new
        current_time = i * dt
        
        # --- Total Potential Force ---
        F_p = _F_p_prof(x, E0, x0, s0, E1, x1, s1)
        
        # --- Phase-Dependent Logic ---
        if current_time < equilibration_time:
            # PHASE 1: EQUILIBRATION (Simple Damping)
            F_total = F_p - gamma_eq * v_half 
            steps_since_last_drop = 0
        else:
            # PHASE 2: METADYNAMICS (Langevin + Bias)
            # F_random = sqrt(2*gamma*T/dt) * Gaussian(0,1)
            random_force = np.random.normal(0, np.sqrt(2 * gamma_mtd * temperature_mtd / dt)) 
            F_bias = _F_bias(x, bias_centers[:num_bias_centers], H_mtd, w_mtd)
            
            # F_total = F_potential + F_dissipative + F_random + F_bias
            F_total = F_p - gamma_mtd * v_half + random_force + F_bias
            
            # Metadynamics Step (Dropping a Gaussian)
            steps_since_last_drop += 1
            if steps_since_last_drop >= tau_G_steps:
                if num_bias_centers < max_gaussians:
                    bias_centers[num_bias_centers] = x
                    num_bias_centers += 1
                    total_bias_added += H_mtd
                steps_since_last_drop = 0
        
        a = F_total / m
        v = v_half + a * (dt / 2.0)
        
        # Store History
        t_history[i] = current_time
        x_history[i] = x
        v_history[i] = v
        bias_energy_history[i] = total_bias_added
        
    return t_history, x_history, v_history, bias_energy_history, bias_centers[:num_bias_centers]


# ====================================================================
# --- SECTION 4: HELPER FUNCTION FOR ANALYSIS ---
# ====================================================================

def moving_average(data, window_size):
    """
    Computes the simple moving average (SMA) of a 1D array.
    """
    if len(data) < window_size:
        # Not enough data points to compute the average over the window
        return np.array(data) 
    # np.convolve is the efficient way to compute a moving average
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


# ====================================================================
# --- SECTION 5: SIMULATION CLASS ---
# ====================================================================

class MetadynamicsSimulation:
    
    def __init__(self, params):
        self.params = params
        
        # System
        self.m = params['m']
        self.dt = params['dt']
        self.x_min = params['x_min']
        self.x_max = params['x_max']
        
        # --- Potential Parameters (6-param structure) ---
        self.potential_params = (
            params['E0'], params['x0'], params['sigma0'],
            params['E1'], params['x1'], params['sigma1']
        )
        
        # Metadynamics
        self.H_mtd = params['H_mtd']
        self.w_mtd = params['w_mtd']
        self.tau_G = params['tau_G']
        self.tau_G_steps = int(self.tau_G / self.dt)
        
        # Equilibration & Langevin
        self.gamma_eq = params.get('gamma_eq', 5.0) 
        self.equilibration_time = params.get('equilibration_time', 100.0) 
        self.gamma_mtd = params.get('gamma_mtd', 1.0)
        self.temperature_mtd = params.get('temperature_mtd', 1.0)
        self.seed = params.get('seed', 42)

    def run(self, total_time, x_initial, v_initial):
        
        total_steps = int(total_time / self.dt)
        print_interval_steps = total_steps // 10
        if print_interval_steps == 0: print_interval_steps = 1
        
        print(f"🔬 Running simulation for {total_time:.1f} time units "
              f"({total_steps} steps)...")
        print(f"  [Phase 1] Equilibration (damping={self.gamma_eq}) until t = {self.equilibration_time}")
        print(f"  [Phase 2] Metadynamics (Langevin T={self.temperature_mtd}, gamma={self.gamma_mtd}) from t > {self.equilibration_time}")
        print("  Simulation is JIT-compiled with Numba. This may take a moment on first run...")

        start_time = time.time()
        
        t_h, x_h, v_h, b_e_h, b_c = run_simulation_loop(
            total_steps, self.dt, self.x_min, self.x_max,
            x_initial, v_initial, self.m, 
            self.gamma_eq, self.equilibration_time, 
            self.gamma_mtd, self.temperature_mtd,
            self.tau_G_steps, self.H_mtd, self.w_mtd,
            self.potential_params,
            print_interval_steps, self.seed
        )
        
        end_time = time.time()
        print(f"✅ Simulation complete. Time taken: {end_time - start_time:.2f} seconds.")
        
        self.results = {
            "t": t_h, "x": x_h, "v": v_h,
            "bias_energy": b_e_h, "bias_centers": b_c
        }
        return self.results

    def plot_results(self):
        """Generates trajectory and bias energy plots."""
        
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        results = self.results
        print(f"\n📊 Plotting standard results...")
        print(f"Total Gaussians dropped: {len(results['bias_centers'])}")
        
        # Plot 1: x vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['x'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'End Equilibration (t={self.equilibration_time})')
        plt.title("Particle Position vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Position x (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot1_name = "plot_1_position_vs_time.png"
        plt.savefig(plot1_name); print(f"Saved: {plot1_name}"); plt.close()
        
        # Plot 2: v vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['v'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'End Equilibration (t={self.equilibration_time})')
        plt.title("Particle Velocity vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Velocity (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot2_name = "plot_2_velocity_vs_time.png"
        plt.savefig(plot2_name); print(f"Saved: {plot2_name}"); plt.close()
        
        # Plot 3: delta_E vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['bias_energy'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'Metadynamics Start (t={self.equilibration_time})')
        plt.title("Total Added Bias Energy ($\Delta E$) vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Cumulative Bias Energy (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot3_name = "plot_3_bias_energy_vs_time.png"
        plt.savefig(plot3_name); print(f"Saved: {plot3_name}"); plt.close()

        # --- Plot 4: Final Energy Landscape ---
        x_grid = np.linspace(self.x_min, self.x_max, 1000)
        
        E_p_grid = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        V_bias_grid = np.array([_V_bias(x, results['bias_centers'], self.H_mtd, self.w_mtd) for x in x_grid])
        
        E_total_filled = E_p_grid + V_bias_grid
        fes_grid = -V_bias_grid
        
        # Shift FES to match the minimum of the original potential for visual comparison
        fes_grid_shifted = fes_grid - (np.min(fes_grid) - np.min(E_p_grid))
        
        plt.figure(figsize=(12, 7))
        plt.title("Final Energy Landscape vs. Configuration (x)")
        plt.xlabel("Position x (units)"); plt.ylabel("Energy (units)")
        plt.plot(x_grid, E_p_grid, 'k-', linewidth=2, label="Original Potential ($E_p$)")
        plt.plot(x_grid, E_total_filled, 'r--', linewidth=2, label="Filled Potential ($E_p + V_{bias}$)")
        plt.plot(x_grid, fes_grid_shifted, 'b-', linewidth=3, alpha=0.7, label="Reconstructed FES ($-V_{bias}$)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6)
        plt.ylim(np.min(E_p_grid) - 5, np.max(E_total_filled) + 5)
        
        plot4_name = "plot_4_final_energy_landscape.png"
        plt.savefig(plot4_name); print(f"Saved: {plot4_name}"); plt.close()

    # ====================================================================
    # --- SECTION 6: EVOLVING PLOT FUNCTION ---
    # ====================================================================
    def plot_evolving_landscape(self):
        """
        Plots a "snapshot" graph showing the evolution of the filled potential
        over time checkpoints.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print("\n📈 Plotting evolving landscape snapshot...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)
        
        x_grid = np.linspace(self.x_min, self.x_max, 1000)
        E_p_grid = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        
        plt.figure(figsize=(12, 7))
        plt.title("Evolution of the Filled Potential ($E_p + V_{bias}$)")
        plt.xlabel("Position x (units)"); plt.ylabel("Energy (units)")
        plt.plot(x_grid, E_p_grid, 'k-', linewidth=3, label="Original Potential ($E_p$)")

        checkpoints = [0.1, 0.3, 0.6, 1.0] # Checkpoints at 10%, 30%, 60%, 100% of drops
        alphas = [0.3, 0.5, 0.7, 1.0]
        linestyles = ['--', ':', '-.', '-'] # Changed for better visual distinction

        E_total_filled_final = None
        for i, frac in enumerate(checkpoints):
            num_drops_to_use = int(total_drops * frac)
            if num_drops_to_use == 0: continue
            
            centers_subset = all_centers[:num_drops_to_use]
            V_bias_subset = np.array([_V_bias(x, centers_subset, self.H_mtd, self.w_mtd) for x in x_grid])
            E_total_filled = E_p_grid + V_bias_subset
            
            if i == len(checkpoints) - 1:
                E_total_filled_final = E_total_filled
            
            plt.plot(x_grid, E_total_filled, color='r', linestyle=linestyles[i],
                     linewidth=2, alpha=alphas[i], label=f"Filled Potential at {frac*100:.0f}% drops")

        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        if E_total_filled_final is not None:
             plt.ylim(np.min(E_p_grid) - 5, np.max(E_total_filled_final) + 5)
        
        plot5_name = "plot_5_evolving_landscape.png"
        plt.savefig(plot5_name); print(f"Saved: {plot5_name}"); plt.close()

    # ====================================================================
    # --- SECTION 7: FORCE CONVERGENCE PLOT (UPDATED WITH ROLLING AVG) ---
    # ====================================================================
    
    def plot_force_convergence(self, monitor_xs, num_checkpoints=500, window_size=50):
        """
        Plots the convergence of the reconstructed force with a Rolling Average 
        for smoothing.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print(f"\n✨ Plotting force convergence (Rolling Avg Window={window_size}) for x = {monitor_xs}...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)

        # Get MTD params
        H_mtd = self.H_mtd
        w_mtd = self.w_mtd

        if total_drops < num_checkpoints:
            num_checkpoints = total_drops
        if total_drops < window_size:
            print(f"Warning: Total drops ({total_drops}) < window size ({window_size}). No smoothing applied.")
        
        # Create checkpoints (time evolution)
        checkpoints = np.linspace(1, total_drops, num=num_checkpoints, dtype=int)
        
        # Data structures
        force_history = {x_val: [] for x_val in monitor_xs}
        time_axis_drops = []

        # --- 1. Calculate Raw Reconstructed Forces over Time ---
        for num_drops in checkpoints:
            centers_subset = all_centers[:num_drops]
            time_axis_drops.append(num_drops)

            for x_val in monitor_xs:
                # F_bias = -d(V_bias)/dx. 
                # The reconstructed potential force F_recon is F_potential ~ -F_bias 
                f_bias_val = _F_bias(x_val, centers_subset, H_mtd, w_mtd)
                f_recon = -f_bias_val 
                force_history[x_val].append(f_recon)

        # --- 2. Plotting ---
        plt.figure(figsize=(14, 8))
        plt.title(f"Convergence of Reconstructed Force (Rolling Avg Window={window_size})")
        plt.xlabel("Time (Number of Gaussians Dropped)")
        plt.ylabel("Reconstructed Force $-F_{bias}$ (units)")

        # Plot True Force (Reference)
        E0, x0, s0, E1, x1, s1 = self.potential_params
        colors = ['b', 'g', 'm'] 
        
        for i, x_val in enumerate(monitor_xs):
            color = colors[i % len(colors)]
            
            # A. Plot True Force (Dashed Horizontal Line)
            true_force = _F_p_prof(x_val, E0, x0, s0, E1, x1, s1)
            plt.axhline(y=true_force, color=color, linestyle='--', alpha=0.6, 
                        label=f"True Force at x={x_val} (target)")

            # B. Plot Raw Data (Faint)
            raw_data = np.array(force_history[x_val])
            plt.plot(time_axis_drops, raw_data, color=color, alpha=0.15, label='_nolegend_')

            # C. Plot Rolling Average (Solid Bold)
            if len(raw_data) >= window_size:
                smoothed_data = moving_average(raw_data, window_size)
                # Slice the time axis to match the length of the smoothed data
                valid_time_axis = time_axis_drops[window_size-1:]
                plt.plot(valid_time_axis, smoothed_data, color=color, linewidth=2.5,
                         label=f"Rolling Avg at x={x_val}")
            else:
                # If smoothing isn't possible, plot the raw data as the main line
                plt.plot(time_axis_drops, raw_data, color=color, linewidth=2.5,
                         label=f"Raw Data at x={x_val} (Window too large)")


        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        
        plot_name = "plot_6_force_convergence_rolling_avg.png"
        plt.savefig(plot_name); print(f"Saved: {plot_name}"); plt.close()


# ====================================================================
# --- RUN THE SIMULATION ---
# ====================================================================

# 1. Define Simulation Parameters
sim_params = {
    'm': 1.0,        # Mass of the particle
    'dt': 0.001,       # Timestep
    'x_min': -3.0,     
    'x_max': 3.0,      
    
    # --- Potential Parameters ---
    # E_p = -E0*Gauss(x0,s0) - E1*Gauss(x1,s1)
    'E0': 15.0,  'x0': -1.0, 'sigma0': 0.4, # Deep well at x=-1
    'E1': 10.0,  'x1': 1.0,  'sigma1': 0.5, # Shallower, wider well at x=1
    
    # Metadynamics
    'H_mtd': 0.1,      # Height of bias Gaussians
    'w_mtd': 0.1,      # Width of bias Gaussians 
    'tau_G': 1.0,      # Time interval between dropping Gaussians
    
    # Equilibration
    'gamma_eq': 5.0,
    'equilibration_time': 100.0,

    # Langevin Thermostat
    'gamma_mtd': 1.0,
    'temperature_mtd': 1.0,
    'seed': 42
}

# 2. Initialize the Simulation
sim = MetadynamicsSimulation(params=sim_params)

# 3. Run the Simulation
# Total time is 20000.0, tau_G is 1.0, so approx 20,000 Gaussians will be dropped.
results = sim.run(
    total_time=2000.0,
    x_initial=-1.0,
    v_initial=0.0
)

# 4. Plot All Standard Results (Saves plots 1-4)
sim.plot_results()

# 5. Plot the Evolving Landscape (Saves plot 5)
sim.plot_evolving_landscape()

# 6. Plot the force convergence with Rolling Average (Saves plot 6)
# monitor_xs: points of interest (e.g., wells and barrier)
# window_size: how many drops to average over (higher = smoother)
sim.plot_force_convergence(monitor_xs=[-1.0, 0.0, 1.0], window_size=50)

🔬 Running simulation for 2000.0 time units (2000000 steps)...
  [Phase 1] Equilibration (damping=5.0) until t = 100.0
  [Phase 2] Metadynamics (Langevin T=1.0, gamma=1.0) from t > 100.0
  Simulation is JIT-compiled with Numba. This may take a moment on first run...
✅ Simulation complete. Time taken: 22.52 seconds.

📊 Plotting standard results...
Total Gaussians dropped: 1900
Saved: plot_1_position_vs_time.png
Saved: plot_2_velocity_vs_time.png
Saved: plot_3_bias_energy_vs_time.png
Saved: plot_4_final_energy_landscape.png

📈 Plotting evolving landscape snapshot...
Saved: plot_5_evolving_landscape.png

✨ Plotting force convergence (Rolling Avg Window=50) for x = [-1.0, 0.0, 1.0]...
Saved: plot_6_force_convergence_rolling_avg.png


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

# --- Numba Imports ---
from numba import jit, njit, prange

# ====================================================================
# --- SECTION 1: POTENTIAL & FORCE (Numba JIT) ---
# ====================================================================

@njit
def _E_p_prof(x, E0, x0, sigma0, E1, x1, sigma1):
    """
    Professor's ground truth potential: E_p = -E0*Gauss(x0,s0) - E1*Gauss(x1,s1)
    """
    well0 = -E0 * np.exp(-(x - x0)**2 / (2 * sigma0**2))
    well1 = -E1 * np.exp(-(x - x1)**2 / (2 * sigma1**2))
    return well0 + well1

@njit
def _F_p_prof(x, E0, x0, sigma0, E1, x1, sigma1):
    """
    Force from the professor's potential: F = -d(E_p)/dx
    """
    f_well0 = E0 * np.exp(-(x - x0)**2 / (2 * sigma0**2)) * (x - x0) / sigma0**2
    f_well1 = E1 * np.exp(-(x - x1)**2 / (2 * sigma1**2)) * (x - x1) / sigma1**2
    return -(f_well0 + f_well1)


# ====================================================================
# --- SECTION 2: METADYNAMICS BIAS FUNCTIONS (Numba JIT) ---
# ====================================================================

@njit
def _V_bias(x, bias_centers, H_mtd, w_mtd):
    """Total bias potential V_bias(x) from all dropped Gaussians"""
    v_b = 0.0
    for i in prange(len(bias_centers)):
        x_c = bias_centers[i]
        v_b += H_mtd * np.exp(-(x - x_c)**2 / (2 * w_mtd**2))
    return v_b

@njit
def _F_bias(x, bias_centers, H_mtd, w_mtd):
    """Force from bias potential: F_bias = -d(V_bias)/dx"""
    f_b = 0.0
    for i in prange(len(bias_centers)):
        x_c = bias_centers[i]
        f_b += H_mtd * np.exp(-(x - x_c)**2 / (2 * w_mtd**2)) * \
               (-(x - x_c) / w_mtd**2)
    return -f_b # F = -dV/dx


# ====================================================================
# --- SECTION 3: MAIN SIMULATION LOOP (Numba JIT) ---
# ====================================================================

@jit(nopython=True) 
def run_simulation_loop(
    total_steps, dt, x_min, x_max,
    x_initial, v_initial, m, 
    gamma_eq, equilibration_time,
    gamma_mtd, temperature_mtd,
    tau_G_steps, H_mtd, w_mtd,
    potential_params, 
    print_interval_steps, seed
):
    """
    This is the core simulation loop, optimized for Numba.
    """
    
    # --- Unpack 6 parameters ---
    E0, x0, s0, E1, x1, s1 = potential_params

    # History arrays
    t_history = np.zeros(total_steps)
    x_history = np.zeros(total_steps)
    v_history = np.zeros(total_steps)
    bias_energy_history = np.zeros(total_steps)
    
    max_gaussians = total_steps // tau_G_steps + 1
    bias_centers = np.zeros(max_gaussians, dtype=np.float64)
    num_bias_centers = 0
    total_bias_added = 0.0
    
    x = x_initial
    v = v_initial
    steps_since_last_drop = 0
    np.random.seed(seed)

    # Initial force calculation
    F_p = _F_p_prof(x, E0, x0, s0, E1, x1, s1)
    
    if 0 < equilibration_time:
        F_total = F_p - gamma_eq * v
    else:
        random_force = np.random.normal(0, np.sqrt(2 * gamma_mtd * temperature_mtd / dt)) 
        F_total = F_p - gamma_mtd * v + random_force + _F_bias(x, bias_centers[:num_bias_centers], H_mtd, w_mtd)
    
    a = F_total / m
    
    # Main Velocity-Verlet Loop
    for i in range(total_steps):
        
        v_half = v + a * (dt / 2.0)
        x_new = x + v_half * dt
        
        # Reflective boundary conditions
        if x_new < x_min:
            x_new = x_min + (x_min - x_new); v_half = -v_half
        elif x_new > x_max:
            x_new = x_max - (x_new - x_max); v_half = -v_half
        
        x = x_new
        current_time = i * dt
        
        # --- Total Potential Force ---
        F_p = _F_p_prof(x, E0, x0, s0, E1, x1, s1)
        
        # --- Phase-Dependent Logic ---
        if current_time < equilibration_time:
            # PHASE 1: EQUILIBRATION
            F_total = F_p - gamma_eq * v_half 
            steps_since_last_drop = 0
        else:
            # PHASE 2: METADYNAMICS
            random_force = np.random.normal(0, np.sqrt(2 * gamma_mtd * temperature_mtd / dt)) 
            F_bias = _F_bias(x, bias_centers[:num_bias_centers], H_mtd, w_mtd)
            F_total = F_p - gamma_mtd * v_half + random_force + F_bias
            
            # Metadynamics Step
            steps_since_last_drop += 1
            if steps_since_last_drop >= tau_G_steps:
                if num_bias_centers < max_gaussians:
                    bias_centers[num_bias_centers] = x
                    num_bias_centers += 1
                    total_bias_added += H_mtd
                steps_since_last_drop = 0
        
        a = F_total / m
        v = v_half + a * (dt / 2.0)
        
        # Store History
        t_history[i] = current_time
        x_history[i] = x
        v_history[i] = v
        bias_energy_history[i] = total_bias_added
        
    return t_history, x_history, v_history, bias_energy_history, bias_centers[:num_bias_centers]


# ====================================================================
# --- SECTION 4: HELPER FUNCTION FOR ANALYSIS ---
# ====================================================================

def moving_average(data, window_size):
    """
    Computes the simple moving average (SMA) of a 1D array.
    """
    if len(data) < window_size:
        return np.array(data) 
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


# ====================================================================
# --- SECTION 5: SIMULATION CLASS ---
# ====================================================================

class MetadynamicsSimulation:
    
    def __init__(self, params):
        self.params = params
        
        # System
        self.m = params['m']
        self.dt = params['dt']
        self.x_min = params['x_min']
        self.x_max = params['x_max']
        
        # Potential Parameters (6-param structure)
        self.potential_params = (
            params['E0'], params['x0'], params['sigma0'],
            params['E1'], params['x1'], params['sigma1']
        )
        
        # Metadynamics
        self.H_mtd = params['H_mtd']
        self.w_mtd = params['w_mtd']
        self.tau_G = params['tau_G']
        self.tau_G_steps = int(self.tau_G / self.dt)
        
        # Equilibration & Langevin
        self.gamma_eq = params.get('gamma_eq', 5.0) 
        self.equilibration_time = params.get('equilibration_time', 100.0) 
        self.gamma_mtd = params.get('gamma_mtd', 1.0)
        self.temperature_mtd = params.get('temperature_mtd', 1.0)
        self.seed = params.get('seed', 42)

    def run(self, total_time, x_initial, v_initial):
        
        total_steps = int(total_time / self.dt)
        print_interval_steps = total_steps // 10
        if print_interval_steps == 0: print_interval_steps = 1
        
        print(f"🔬 Running simulation for {total_time:.1f} time units "
              f"({total_steps} steps)...")
        print("  Simulation is JIT-compiled with Numba...")

        start_time = time.time()
        
        t_h, x_h, v_h, b_e_h, b_c = run_simulation_loop(
            total_steps, self.dt, self.x_min, self.x_max,
            x_initial, v_initial, self.m, 
            self.gamma_eq, self.equilibration_time, 
            self.gamma_mtd, self.temperature_mtd,
            self.tau_G_steps, self.H_mtd, self.w_mtd,
            self.potential_params,
            print_interval_steps, self.seed
        )
        
        end_time = time.time()
        print(f"✅ Simulation complete. Time taken: {end_time - start_time:.2f} seconds.")
        
        self.results = {
            "t": t_h, "x": x_h, "v": v_h,
            "bias_energy": b_e_h, "bias_centers": b_c
        }
        return self.results

    def plot_results(self):
        """Generates standard plots."""
        
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        results = self.results
        print(f"\n📊 Plotting standard results...")
        
        # Plot 1: x vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['x'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'End Equilibration (t={self.equilibration_time})')
        plt.title("Particle Position vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Position x (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot1_name = "plot_1_position_vs_time.png"
        plt.savefig(plot1_name); plt.close(); print(f"Saved: {plot1_name}")
        
        # Plot 2: v vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['v'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'End Equilibration (t={self.equilibration_time})')
        plt.title("Particle Velocity vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Velocity (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot2_name = "plot_2_velocity_vs_time.png"
        plt.savefig(plot2_name); plt.close(); print(f"Saved: {plot2_name}")
        
        # Plot 3: delta_E vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['bias_energy'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'Metadynamics Start (t={self.equilibration_time})')
        plt.title("Total Added Bias Energy ($\Delta E$) vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Cumulative Bias Energy (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot3_name = "plot_3_bias_energy_vs_time.png"
        plt.savefig(plot3_name); plt.close(); print(f"Saved: {plot3_name}")

        # Plot 4: Final Energy Landscape
        x_grid = np.linspace(self.x_min, self.x_max, 1000)
        E_p_grid = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        V_bias_grid = np.array([_V_bias(x, results['bias_centers'], self.H_mtd, self.w_mtd) for x in x_grid])
        
        E_total_filled = E_p_grid + V_bias_grid
        fes_grid = -V_bias_grid
        fes_grid_shifted = fes_grid - (np.min(fes_grid) - np.min(E_p_grid))
        
        plt.figure(figsize=(12, 7))
        plt.title("Final Energy Landscape vs. Configuration (x)")
        plt.xlabel("Position x (units)"); plt.ylabel("Energy (units)")
        plt.plot(x_grid, E_p_grid, 'k-', linewidth=2, label="Original Potential ($E_p$)")
        plt.plot(x_grid, E_total_filled, 'r--', linewidth=2, label="Filled Potential ($E_p + V_{bias}$)")
        plt.plot(x_grid, fes_grid_shifted, 'b-', linewidth=3, alpha=0.7, label="Reconstructed FES ($-V_{bias}$)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6)
        plt.ylim(np.min(E_p_grid) - 5, np.max(E_total_filled) + 5)
        plot4_name = "plot_4_final_energy_landscape.png"
        plt.savefig(plot4_name); plt.close(); print(f"Saved: {plot4_name}")

    # ====================================================================
    # --- SECTION 6: EVOLVING PLOT FUNCTION ---
    # ====================================================================
    def plot_evolving_landscape(self):
        """
        Plots a "snapshot" graph showing the evolution of the filled potential.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print("\n📈 Plotting evolving landscape snapshot...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)
        
        x_grid = np.linspace(self.x_min, self.x_max, 1000)
        E_p_grid = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        
        plt.figure(figsize=(12, 7))
        plt.title("Evolution of the Filled Potential ($E_p + V_{bias}$)")
        plt.xlabel("Position x (units)"); plt.ylabel("Energy (units)")
        plt.plot(x_grid, E_p_grid, 'k-', linewidth=3, label="Original Potential ($E_p$)")

        checkpoints = [0.1, 0.3, 0.6, 1.0] 
        alphas = [0.3, 0.5, 0.7, 1.0]
        linestyles = ['--', ':', '-.', '-'] 

        E_total_filled_final = None
        for i, frac in enumerate(checkpoints):
            num_drops_to_use = int(total_drops * frac)
            if num_drops_to_use == 0: continue
            
            centers_subset = all_centers[:num_drops_to_use]
            V_bias_subset = np.array([_V_bias(x, centers_subset, self.H_mtd, self.w_mtd) for x in x_grid])
            E_total_filled = E_p_grid + V_bias_subset
            
            if i == len(checkpoints) - 1:
                E_total_filled_final = E_total_filled
            
            plt.plot(x_grid, E_total_filled, color='r', linestyle=linestyles[i],
                     linewidth=2, alpha=alphas[i], label=f"Filled Potential at {frac*100:.0f}% drops")

        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        if E_total_filled_final is not None:
             plt.ylim(np.min(E_p_grid) - 5, np.max(E_total_filled_final) + 5)
        
        plot5_name = "plot_5_evolving_landscape.png"
        plt.savefig(plot5_name); plt.close(); print(f"Saved: {plot5_name}")

    # ====================================================================
    # --- SECTION 7: FORCE CONVERGENCE PLOT ---
    # ====================================================================
    
    def plot_force_convergence(self, monitor_xs, num_checkpoints=500, window_size=50):
        """
        Plots the convergence of the reconstructed force at specific points.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print(f"\n✨ Plotting point-wise force convergence (Window={window_size})...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)

        H_mtd = self.H_mtd
        w_mtd = self.w_mtd

        if total_drops < num_checkpoints:
            num_checkpoints = total_drops
        
        checkpoints = np.linspace(1, total_drops, num=num_checkpoints, dtype=int)
        
        force_history = {x_val: [] for x_val in monitor_xs}
        time_axis_drops = []

        # Calculate Raw Reconstructed Forces over Time
        for num_drops in checkpoints:
            centers_subset = all_centers[:num_drops]
            time_axis_drops.append(num_drops)

            for x_val in monitor_xs:
                f_bias_val = _F_bias(x_val, centers_subset, H_mtd, w_mtd)
                f_recon = -f_bias_val 
                force_history[x_val].append(f_recon)

        # Plotting
        plt.figure(figsize=(14, 8))
        plt.title(f"Convergence of Reconstructed Force (Rolling Avg Window={window_size})")
        plt.xlabel("Time (Number of Gaussians Dropped)")
        plt.ylabel("Reconstructed Force $-F_{bias}$ (units)")

        E0, x0, s0, E1, x1, s1 = self.potential_params
        colors = ['b', 'g', 'm'] 
        
        for i, x_val in enumerate(monitor_xs):
            color = colors[i % len(colors)]
            
            # A. Plot True Force (Target)
            true_force = _F_p_prof(x_val, E0, x0, s0, E1, x1, s1)
            plt.axhline(y=true_force, color=color, linestyle='--', alpha=0.6, 
                        label=f"True Force at x={x_val} (target)")

            # B. Plot Raw Data (Faint)
            raw_data = np.array(force_history[x_val])
            plt.plot(time_axis_drops, raw_data, color=color, alpha=0.15, label='_nolegend_')

            # C. Plot Rolling Average (Solid Bold)
            if len(raw_data) >= window_size:
                smoothed_data = moving_average(raw_data, window_size)
                valid_time_axis = time_axis_drops[window_size-1:]
                plt.plot(valid_time_axis, smoothed_data, color=color, linewidth=2.5,
                         label=f"Rolling Avg at x={x_val}")

        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plot_name = "plot_6_force_convergence_rolling_avg.png"
        plt.savefig(plot_name); plt.close(); print(f"Saved: {plot_name}")

    # ====================================================================
    # --- SECTION 8: MAX FORCE CONVERGENCE PLOT (NEW) ---
    # ====================================================================
    
    def plot_max_force_convergence(self, num_checkpoints=500, window_size=50):
        """
        Calculates and plots the convergence of the maximum *absolute* reconstructed force (Max|F_recon|) across the domain vs. time.
        Convergence criterion for Metadynamics is Max|F_recon| -> 0.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print(f"\n⚡ Plotting Maximum Absolute Force Convergence (Window={window_size})...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)

        H_mtd = self.H_mtd
        w_mtd = self.w_mtd
        
        # Grid for sampling the force (finer grid = more accurate max)
        x_grid = np.linspace(self.x_min, self.x_max, 2000) 

        if total_drops < num_checkpoints:
            num_checkpoints = total_drops
        
        checkpoints = np.linspace(1, total_drops, num=num_checkpoints, dtype=int)
        
        max_abs_force_history = []
        time_axis_drops = []

        # --- Calculate Max Force at Checkpoints ---
        for num_drops in checkpoints:
            centers_subset = all_centers[:num_drops]
            time_axis_drops.append(num_drops)

            # F_recon = -F_bias. We want the absolute maximum of this force.
            F_recon_grid = np.array([-_F_bias(x, centers_subset, H_mtd, w_mtd) for x in x_grid])
            
            max_abs_force = np.max(np.abs(F_recon_grid))
            max_abs_force_history.append(max_abs_force)

        # --- Apply Rolling Average ---
        raw_data = np.array(max_abs_force_history)
        
        # --- Create the Plot ---
        plt.figure(figsize=(12, 7))
        plt.title(f"Convergence of Max Reconstructed Force (Max $|-F_{{bias}}|$ vs. Time)")
        plt.xlabel("Time (Number of Gaussians Dropped)")
        plt.ylabel("Maximum Absolute Force Magnitude (units)")
        
        # Target is for the max force to approach zero
        plt.axhline(y=0, color='r', linestyle='--', label="Target Convergence (Max Force = 0)")

        # Plot Raw Data (Faint)
        plt.plot(time_axis_drops, raw_data, 'b-', alpha=0.15, label='_nolegend_')

        # Plot Rolling Average (Solid Bold)
        if len(raw_data) >= window_size:
            smoothed_data = moving_average(raw_data, window_size)
            valid_time_axis = time_axis_drops[window_size-1:]
            plt.plot(valid_time_axis, smoothed_data, 'b-', linewidth=2.5,
                     label=f"Rolling Avg (Window={window_size})")
        else:
             plt.plot(time_axis_drops, raw_data, 'b-', linewidth=2.5,
                     label="Raw Max Force (Window too large)")

        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plot_name = "plot_7_max_force_convergence.png"
        plt.savefig(plot_name); plt.close(); print(f"Saved: {plot_name}")


# ====================================================================
# --- RUN THE SIMULATION ---
# ====================================================================

# 1. Define Simulation Parameters
sim_params = {
    'm': 1.0,        'dt': 0.001,       
    'x_min': -3.0,     'x_max': 3.0,      
    
    # Potential Parameters
    'E0': 15.0,  'x0': -1.0, 'sigma0': 0.4, 
    'E1': 10.0,  'x1': 1.0,  'sigma1': 0.5, 
    
    # Metadynamics
    'H_mtd': 0.1,      'w_mtd': 0.1,      
    'tau_G': 1.0,      
    
    # Equilibration
    'gamma_eq': 5.0, 'equilibration_time': 100.0,

    # Langevin Thermostat
    'gamma_mtd': 1.0, 'temperature_mtd': 1.0,
    'seed': 42
}

# 2. Initialize the Simulation
sim = MetadynamicsSimulation(params=sim_params)

# 3. Run the Simulation
results = sim.run(
    total_time=2000.0,
    x_initial=-1.0,
    v_initial=0.0
)

# 4. Plot All Standard Results (Saves plots 1-4)
sim.plot_results()

# 5. Plot the Evolving Landscape (Saves plot 5)
sim.plot_evolving_landscape()

# 6. Plot the point-wise force convergence (Saves plot 6)
sim.plot_force_convergence(monitor_xs=[-1.0, 0.0, 1.0], window_size=50)

# 7. NEW: Plot the maximum absolute force convergence (Saves plot 7)
# This is the key metric for checking if the barrier is "fully filled."
sim.plot_max_force_convergence(window_size=50)

🔬 Running simulation for 2000.0 time units (2000000 steps)...
  Simulation is JIT-compiled with Numba...
✅ Simulation complete. Time taken: 19.88 seconds.

📊 Plotting standard results...
Saved: plot_1_position_vs_time.png
Saved: plot_2_velocity_vs_time.png
Saved: plot_3_bias_energy_vs_time.png
Saved: plot_4_final_energy_landscape.png

📈 Plotting evolving landscape snapshot...
Saved: plot_5_evolving_landscape.png

✨ Plotting point-wise force convergence (Window=50)...
Saved: plot_6_force_convergence_rolling_avg.png

⚡ Plotting Maximum Absolute Force Convergence (Window=50)...
Saved: plot_7_max_force_convergence.png


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import time

# --- Numba Imports ---
from numba import jit, njit, prange

# ====================================================================
# --- SECTION 1: POTENTIAL & FORCE (Numba JIT) ---
# ====================================================================

@njit
def _E_p_prof(x, E0, x0, sigma0, E1, x1, sigma1):
    """
    Professor's ground truth potential: E_p = -E0*Gauss(x0,s0) - E1*Gauss(x1,s1)
    """
    well0 = -E0 * np.exp(-(x - x0)**2 / (2 * sigma0**2))
    well1 = -E1 * np.exp(-(x - x1)**2 / (2 * sigma1**2))
    return well0 + well1

@njit
def _F_p_prof(x, E0, x0, sigma0, E1, x1, sigma1):
    """
    Force from the professor's potential: F = -d(E_p)/dx
    """
    f_well0 = E0 * np.exp(-(x - x0)**2 / (2 * sigma0**2)) * (x - x0) / sigma0**2
    f_well1 = E1 * np.exp(-(x - x1)**2 / (2 * sigma1**2)) * (x - x1) / sigma1**2
    return -(f_well0 + f_well1)


# ====================================================================
# --- SECTION 2: METADYNAMICS BIAS FUNCTIONS (Numba JIT) ---
# ====================================================================

@njit
def _V_bias(x, bias_centers, H_mtd, w_mtd):
    """Total bias potential V_bias(x) from all dropped Gaussians"""
    v_b = 0.0
    for i in prange(len(bias_centers)):
        x_c = bias_centers[i]
        v_b += H_mtd * np.exp(-(x - x_c)**2 / (2 * w_mtd**2))
    return v_b

@njit
def _F_bias(x, bias_centers, H_mtd, w_mtd):
    """Force from bias potential: F_bias = -d(V_bias)/dx"""
    f_b = 0.0
    for i in prange(len(bias_centers)):
        x_c = bias_centers[i]
        f_b += H_mtd * np.exp(-(x - x_c)**2 / (2 * w_mtd**2)) * \
               (-(x - x_c) / w_mtd**2)
    return -f_b # F = -dV/dx


# ====================================================================
# --- SECTION 3: MAIN SIMULATION LOOP (Numba JIT) ---
# ====================================================================

@jit(nopython=True) 
def run_simulation_loop(
    total_steps, dt, x_min, x_max,
    x_initial, v_initial, m, 
    gamma_eq, equilibration_time,
    gamma_mtd, temperature_mtd,
    tau_G_steps, H_mtd, w_mtd,
    potential_params, 
    print_interval_steps, seed
):
    """
    This is the core simulation loop, optimized for Numba.
    """
    
    E0, x0, s0, E1, x1, s1 = potential_params

    t_history = np.zeros(total_steps)
    x_history = np.zeros(total_steps)
    v_history = np.zeros(total_steps)
    bias_energy_history = np.zeros(total_steps)
    
    max_gaussians = total_steps // tau_G_steps + 1
    bias_centers = np.zeros(max_gaussians, dtype=np.float64)
    num_bias_centers = 0
    total_bias_added = 0.0
    
    x = x_initial
    v = v_initial
    steps_since_last_drop = 0
    np.random.seed(seed)

    F_p = _F_p_prof(x, E0, x0, s0, E1, x1, s1)
    
    if 0 < equilibration_time:
        F_total = F_p - gamma_eq * v
    else:
        random_force = np.random.normal(0, np.sqrt(2 * gamma_mtd * temperature_mtd / dt)) 
        F_total = F_p - gamma_mtd * v + random_force + _F_bias(x, bias_centers[:num_bias_centers], H_mtd, w_mtd)
    
    a = F_total / m
    
    for i in range(total_steps):
        
        v_half = v + a * (dt / 2.0)
        x_new = x + v_half * dt
        
        # Reflective boundary conditions
        if x_new < x_min:
            x_new = x_min + (x_min - x_new); v_half = -v_half
        elif x_new > x_max:
            x_new = x_max - (x_new - x_max); v_half = -v_half
        
        x = x_new
        current_time = i * dt
        
        F_p = _F_p_prof(x, E0, x0, s0, E1, x1, s1)
        
        if current_time < equilibration_time:
            F_total = F_p - gamma_eq * v_half 
            steps_since_last_drop = 0
        else:
            random_force = np.random.normal(0, np.sqrt(2 * gamma_mtd * temperature_mtd / dt)) 
            F_bias = _F_bias(x, bias_centers[:num_bias_centers], H_mtd, w_mtd)
            F_total = F_p - gamma_mtd * v_half + random_force + F_bias
            
            steps_since_last_drop += 1
            if steps_since_last_drop >= tau_G_steps:
                if num_bias_centers < max_gaussians:
                    bias_centers[num_bias_centers] = x
                    num_bias_centers += 1
                    total_bias_added += H_mtd
                steps_since_last_drop = 0
        
        a = F_total / m
        v = v_half + a * (dt / 2.0)
        
        t_history[i] = current_time
        x_history[i] = x
        v_history[i] = v
        bias_energy_history[i] = total_bias_added
        
    return t_history, x_history, v_history, bias_energy_history, bias_centers[:num_bias_centers]


# ====================================================================
# --- SECTION 4: HELPER FUNCTION FOR ANALYSIS ---
# ====================================================================

def moving_average(data, window_size):
    """
    Computes the simple moving average (SMA) of a 1D array.
    """
    if len(data) < window_size:
        return np.array(data) 
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


# ====================================================================
# --- SECTION 5: SIMULATION CLASS ---
# ====================================================================

class MetadynamicsSimulation:
    
    def __init__(self, params):
        self.params = params
        
        self.m = params['m']
        self.dt = params['dt']
        self.x_min = params['x_min']
        self.x_max = params['x_max']
        
        self.potential_params = (
            params['E0'], params['x0'], params['sigma0'],
            params['E1'], params['x1'], params['sigma1']
        )
        
        self.H_mtd = params['H_mtd']
        self.w_mtd = params['w_mtd']
        self.tau_G = params['tau_G']
        self.tau_G_steps = int(self.tau_G / self.dt)
        
        self.gamma_eq = params.get('gamma_eq', 5.0) 
        self.equilibration_time = params.get('equilibration_time', 100.0) 
        self.gamma_mtd = params.get('gamma_mtd', 1.0)
        self.temperature_mtd = params.get('temperature_mtd', 1.0)
        self.seed = params.get('seed', 42)

    def run(self, total_time, x_initial, v_initial):
        
        total_steps = int(total_time / self.dt)
        print_interval_steps = total_steps // 10
        if print_interval_steps == 0: print_interval_steps = 1
        
        print(f"🔬 Running simulation for {total_time:.1f} time units "
              f"({total_steps} steps)...")

        start_time = time.time()
        
        t_h, x_h, v_h, b_e_h, b_c = run_simulation_loop(
            total_steps, self.dt, self.x_min, self.x_max,
            x_initial, v_initial, self.m, 
            self.gamma_eq, self.equilibration_time, 
            self.gamma_mtd, self.temperature_mtd,
            self.tau_G_steps, self.H_mtd, self.w_mtd,
            self.potential_params,
            print_interval_steps, self.seed
        )
        
        end_time = time.time()
        print(f"✅ Simulation complete. Time taken: {end_time - start_time:.2f} seconds.")
        
        self.results = {
            "t": t_h, "x": x_h, "v": v_h,
            "bias_energy": b_e_h, "bias_centers": b_c
        }
        return self.results

    def plot_results(self):
        """Generates standard plots."""
        
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        results = self.results
        print(f"\n📊 Plotting standard results...")
        
        # Plot 1: x vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['x'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'End Equilibration (t={self.equilibration_time})')
        plt.title("Particle Position vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Position x (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot1_name = "plot_1_position_vs_time.png"
        plt.savefig(plot1_name); plt.close(); print(f"Saved: {plot1_name}")
        
        # Plot 2: v vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['v'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'End Equilibration (t={self.equilibration_time})')
        plt.title("Particle Velocity vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Velocity (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot2_name = "plot_2_velocity_vs_time.png"
        plt.savefig(plot2_name); plt.close(); print(f"Saved: {plot2_name}")
        
        # Plot 3: delta_E vs t
        plt.figure(figsize=(12, 5)); plt.plot(results['t'], results['bias_energy'])
        plt.axvline(x=self.equilibration_time, color='r', linestyle='--', label=f'Metadynamics Start (t={self.equilibration_time})')
        plt.title("Total Added Bias Energy ($\Delta E$) vs. Time"); plt.xlabel("Time (units)"); plt.ylabel("Cumulative Bias Energy (units)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plot3_name = "plot_3_bias_energy_vs_time.png"
        plt.savefig(plot3_name); plt.close(); print(f"Saved: {plot3_name}")

        # Plot 4: Final Energy Landscape
        x_grid = np.linspace(self.x_min, self.x_max, 1000)
        E_p_grid = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        V_bias_grid = np.array([_V_bias(x, results['bias_centers'], self.H_mtd, self.w_mtd) for x in x_grid])
        
        E_total_filled = E_p_grid + V_bias_grid
        fes_grid = -V_bias_grid
        fes_grid_shifted = fes_grid - (np.min(fes_grid) - np.min(E_p_grid))
        
        plt.figure(figsize=(12, 7))
        plt.title("Final Energy Landscape vs. Configuration (x)")
        plt.xlabel("Position x (units)"); plt.ylabel("Energy (units)")
        plt.plot(x_grid, E_p_grid, 'k-', linewidth=2, label="Original Potential ($E_p$)")
        plt.plot(x_grid, E_total_filled, 'r--', linewidth=2, label="Filled Potential ($E_p + V_{bias}$)")
        plt.plot(x_grid, fes_grid_shifted, 'b-', linewidth=3, alpha=0.7, label="Reconstructed FES ($-V_{bias}$)")
        plt.legend(); plt.grid(True, linestyle='--', alpha=0.6)
        plt.ylim(np.min(E_p_grid) - 5, np.max(E_total_filled) + 5)
        plot4_name = "plot_4_final_energy_landscape.png"
        plt.savefig(plot4_name); plt.close(); print(f"Saved: {plot4_name}")

    # ====================================================================
    # --- SECTION 6: EVOLVING PLOT FUNCTION ---
    # ====================================================================
    def plot_evolving_landscape(self):
        """
        Plots a "snapshot" graph showing the evolution of the filled potential.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print("\n📈 Plotting evolving landscape snapshot...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)
        
        x_grid = np.linspace(self.x_min, self.x_max, 1000)
        E_p_grid = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        
        plt.figure(figsize=(12, 7))
        plt.title("Evolution of the Filled Potential ($E_p + V_{bias}$)")
        plt.xlabel("Position x (units)"); plt.ylabel("Energy (units)")
        plt.plot(x_grid, E_p_grid, 'k-', linewidth=3, label="Original Potential ($E_p$)")

        checkpoints = [0.1, 0.3, 0.6, 1.0] 
        alphas = [0.3, 0.5, 0.7, 1.0]
        linestyles = ['--', ':', '-.', '-'] 

        E_total_filled_final = None
        for i, frac in enumerate(checkpoints):
            num_drops_to_use = int(total_drops * frac)
            if num_drops_to_use == 0: continue
            
            centers_subset = all_centers[:num_drops_to_use]
            V_bias_subset = np.array([_V_bias(x, centers_subset, self.H_mtd, self.w_mtd) for x in x_grid])
            E_total_filled = E_p_grid + V_bias_subset
            
            if i == len(checkpoints) - 1:
                E_total_filled_final = E_total_filled
            
            plt.plot(x_grid, E_total_filled, color='r', linestyle=linestyles[i],
                     linewidth=2, alpha=alphas[i], label=f"Filled Potential at {frac*100:.0f}% drops")

        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        if E_total_filled_final is not None:
             plt.ylim(np.min(E_p_grid) - 5, np.max(E_total_filled_final) + 5)
        
        plot5_name = "plot_5_evolving_landscape.png"
        plt.savefig(plot5_name); plt.close(); print(f"Saved: {plot5_name}")

    # ====================================================================
    # --- SECTION 7: POINT-WISE FORCE CONVERGENCE PLOT ---
    # ====================================================================
    
    def plot_force_convergence(self, monitor_xs, num_checkpoints=500, window_size=50):
        """
        Plots the convergence of the reconstructed force at specific points.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print(f"\n✨ Plotting point-wise force convergence (Window={window_size})...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)

        H_mtd = self.H_mtd
        w_mtd = self.w_mtd

        if total_drops < num_checkpoints:
            num_checkpoints = total_drops
        
        checkpoints = np.linspace(1, total_drops, num=num_checkpoints, dtype=int)
        
        force_history = {x_val: [] for x_val in monitor_xs}
        time_axis_drops = []

        # Calculate Raw Reconstructed Forces over Time
        for num_drops in checkpoints:
            centers_subset = all_centers[:num_drops]
            time_axis_drops.append(num_drops)

            for x_val in monitor_xs:
                f_bias_val = _F_bias(x_val, centers_subset, H_mtd, w_mtd)
                f_recon = -f_bias_val 
                force_history[x_val].append(f_recon)

        # Plotting
        plt.figure(figsize=(14, 8))
        plt.title(f"Convergence of Reconstructed Force (Rolling Avg Window={window_size})")
        plt.xlabel("Time (Number of Gaussians Dropped)")
        plt.ylabel("Reconstructed Force $-F_{bias}$ (units)")

        E0, x0, s0, E1, x1, s1 = self.potential_params
        colors = ['b', 'g', 'm'] 
        
        for i, x_val in enumerate(monitor_xs):
            color = colors[i % len(colors)]
            
            # A. Plot True Force (Target)
            true_force = _F_p_prof(x_val, E0, x0, s0, E1, x1, s1)
            plt.axhline(y=true_force, color=color, linestyle='--', alpha=0.6, 
                        label=f"True Force at x={x_val} (target)")

            # B. Plot Raw Data (Faint)
            raw_data = np.array(force_history[x_val])
            plt.plot(time_axis_drops, raw_data, color=color, alpha=0.15, label='_nolegend_')

            # C. Plot Rolling Average (Solid Bold)
            if len(raw_data) >= window_size:
                smoothed_data = moving_average(raw_data, window_size)
                valid_time_axis = time_axis_drops[window_size-1:]
                plt.plot(valid_time_axis, smoothed_data, color=color, linewidth=2.5,
                         label=f"Rolling Avg at x={x_val}")

        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plot_name = "plot_6_force_convergence_rolling_avg.png"
        plt.savefig(plot_name); plt.close(); print(f"Saved: {plot_name}")

    # ====================================================================
    # --- SECTION 8: MAX FORCE CONVERGENCE PLOT ---
    # ====================================================================
    
    def plot_max_force_convergence(self, num_checkpoints=500, window_size=50):
        """
        Calculates and plots the convergence of the maximum *absolute* reconstructed force (Max|F_recon|) across the domain vs. time.
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print(f"\n⚡ Plotting Maximum Absolute Force Convergence (Window={window_size})...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)

        H_mtd = self.H_mtd
        w_mtd = self.w_mtd
        
        x_grid = np.linspace(self.x_min, self.x_max, 2000) 

        if total_drops < num_checkpoints:
            num_checkpoints = total_drops
        
        checkpoints = np.linspace(1, total_drops, num=num_checkpoints, dtype=int)
        
        max_abs_force_history = []
        time_axis_drops = []

        # Calculate Max Force at Checkpoints
        for num_drops in checkpoints:
            centers_subset = all_centers[:num_drops]
            time_axis_drops.append(num_drops)

            F_recon_grid = np.array([-_F_bias(x, centers_subset, H_mtd, w_mtd) for x in x_grid])
            
            max_abs_force = np.max(np.abs(F_recon_grid))
            max_abs_force_history.append(max_abs_force)

        # Apply Rolling Average
        raw_data = np.array(max_abs_force_history)
        
        # Create the Plot
        plt.figure(figsize=(12, 7))
        plt.title(f"Convergence of Max Reconstructed Force (Max $|-F_{{bias}}|$ vs. Time)")
        plt.xlabel("Time (Number of Gaussians Dropped)")
        plt.ylabel("Maximum Absolute Force Magnitude (units)")
        
        plt.axhline(y=0, color='r', linestyle='--', label="Target Convergence (Max Force = 0)")

        # Plot Raw Data (Faint)
        plt.plot(time_axis_drops, raw_data, 'b-', alpha=0.15, label='_nolegend_')

        # Plot Rolling Average (Solid Bold)
        if len(raw_data) >= window_size:
            smoothed_data = moving_average(raw_data, window_size)
            valid_time_axis = time_axis_drops[window_size-1:]
            plt.plot(valid_time_axis, smoothed_data, 'b-', linewidth=2.5,
                     label=f"Rolling Avg (Window={window_size})")
        else:
             plt.plot(time_axis_drops, raw_data, 'b-', linewidth=2.5,
                     label="Raw Max Force (Window too large)")

        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plot_name = "plot_7_max_force_convergence.png"
        plt.savefig(plot_name); plt.close(); print(f"Saved: {plot_name}")
        
    # ====================================================================
    # --- SECTION 9: ENERGY RECONSTRUCTION ERROR PLOT (NEW) ---
    # ====================================================================

    def plot_energy_reconstruction_error(self, region_bounds, num_checkpoints=500, window_size=50):
        """
        Calculates and plots the Mean Absolute Error (MAE) of the reconstructed FES 
        relative to the true potential in specified regions over time.
        
        Args:
            region_bounds (list of tuples): [(x_start1, x_end1), (x_start2, x_end2), ...]
        """
        if not hasattr(self, 'results'):
            print("No results to plot. Run .run() first.")
            return

        print(f"\n📏 Plotting MAE of FES Reconstruction (Window={window_size})...")
        results = self.results
        all_centers = results['bias_centers']
        total_drops = len(all_centers)

        H_mtd = self.H_mtd
        w_mtd = self.w_mtd
        
        if total_drops < num_checkpoints:
            num_checkpoints = total_drops
        
        checkpoints = np.linspace(1, total_drops, num=num_checkpoints, dtype=int)
        
        # Grid for calculating FES and E_p, ensure it covers all specified regions
        x_grid = np.linspace(self.x_min, self.x_max, 2000) 
        E_p_grid_full = np.array([_E_p_prof(x, *self.potential_params) for x in x_grid])
        
        # Find the absolute minimum of the true potential for FES alignment
        E_p_min = np.min(E_p_grid_full)
        
        mae_history = {i: [] for i in range(len(region_bounds))}
        time_axis_drops = []

        # --- Calculate MAE at Checkpoints ---
        for num_drops in checkpoints:
            centers_subset = all_centers[:num_drops]
            time_axis_drops.append(num_drops)

            # 1. Calculate the current full FES (-V_bias)
            V_bias_grid_full = np.array([_V_bias(x, centers_subset, H_mtd, w_mtd) for x in x_grid])
            FES_recon_full = -V_bias_grid_full

            # 2. Align the FES: FES_aligned = FES_recon - (min(FES_recon) - min(E_p))
            # The reconstructed FES minimum should match the true potential minimum
            if len(FES_recon_full) > 0:
                FES_min = np.min(FES_recon_full)
                FES_aligned_full = FES_recon_full - (FES_min - E_p_min)
            else:
                FES_aligned_full = np.zeros_like(E_p_grid_full)
            
            # 3. Calculate MAE for each region
            for i, (x_start, x_end) in enumerate(region_bounds):
                # Filter grid data to the specified region
                idx = np.where((x_grid >= x_start) & (x_grid <= x_end))
                
                E_p_region = E_p_grid_full[idx]
                FES_region = FES_aligned_full[idx]
                
                if len(E_p_region) > 0:
                    # MAE = mean(|FES_aligned - E_p|)
                    mae = np.mean(np.abs(FES_region - E_p_region))
                    mae_history[i].append(mae)
                else:
                    mae_history[i].append(np.nan) # Append NaN if region is empty

        # --- Create the Plot ---
        plt.figure(figsize=(12, 7))
        plt.title(f"Mean Absolute Error (MAE) of FES Reconstruction")
        plt.xlabel("Time (Number of Gaussians Dropped)")
        plt.ylabel("Mean Absolute Error (Energy Units)")

        colors = ['b', 'g', 'r', 'c']
        
        for i, (x_start, x_end) in enumerate(region_bounds):
            color = colors[i % len(colors)]
            raw_data = np.array(mae_history[i])
            
            # Remove NaNs that might occur if regions were defined outside the grid
            valid_idx = ~np.isnan(raw_data)
            raw_data_valid = raw_data[valid_idx]
            time_axis_valid = np.array(time_axis_drops)[valid_idx]

            if len(raw_data_valid) == 0:
                 continue
                 
            # A. Plot Raw Data (Faint)
            plt.plot(time_axis_valid, raw_data_valid, color=color, alpha=0.15, label='_nolegend_')

            # B. Plot Rolling Average (Solid Bold)
            if len(raw_data_valid) >= window_size:
                smoothed_data = moving_average(raw_data_valid, window_size)
                valid_time_axis = time_axis_valid[window_size-1:]
                plt.plot(valid_time_axis, smoothed_data, color=color, linewidth=2.5,
                         label=f"Avg Error in [{x_start}, {x_end}]")
            else:
                # Plot raw data if smoothing window is too large
                 plt.plot(time_axis_valid, raw_data_valid, color=color, linewidth=1.5,
                         label=f"Error in [{x_start}, {x_end}] (Raw)")

        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        
        plot_name = "plot_8_fes_mae_convergence.png"
        plt.savefig(plot_name); plt.close(); print(f"Saved: {plot_name}")


# ====================================================================
# --- RUN THE SIMULATION ---
# ====================================================================

# 1. Define Simulation Parameters
sim_params = {
    'm': 1.0,        'dt': 0.001,       
    'x_min': -3.0,     'x_max': 3.0,      
    
    # Potential Parameters
    'E0': 15.0,  'x0': -1.0, 'sigma0': 0.4, 
    'E1': 10.0,  'x1': 1.0,  'sigma1': 0.5, 
    
    # Metadynamics
    'H_mtd': 0.1,      'w_mtd': 0.1,      
    'tau_G': 1.0,      
    
    # Equilibration
    'gamma_eq': 5.0, 'equilibration_time': 100.0,

    # Langevin Thermostat
    'gamma_mtd': 1.0, 'temperature_mtd': 1.0,
    'seed': 42
}

# 2. Initialize the Simulation
sim = MetadynamicsSimulation(params=sim_params)

# 3. Run the Simulation
results = sim.run(
    total_time=2000.0,
    x_initial=-1.0,
    v_initial=0.0
)

# 4. Plot All Standard Results (Saves plots 1-4)
sim.plot_results()

# 5. Plot the Evolving Landscape (Saves plot 5)
sim.plot_evolving_landscape()

# 6. Plot the point-wise force convergence (Saves plot 6)
sim.plot_force_convergence(monitor_xs=[-1.0, 0.0, 1.0], window_size=50)

# 7. Plot the maximum absolute force convergence (Saves plot 7)
sim.plot_max_force_convergence(window_size=50) 

# 8. NEW: Plot the Mean Absolute Error (MAE) for the FES reconstruction (Saves plot 8)
# We define regions around the well centers x=-1.0 and x=1.0.
# Region 1: [-1.5, -0.5] (around well x0=-1.0)
# Region 2: [0.5, 1.5] (around well x1=1.0)
# MAE should approach zero in these regions if the FES is accurate.
monitoring_regions = [
    (-1.5, -0.5), # Deep well region
    (0.5, 1.5)    # Shallow well region
]
sim.plot_energy_reconstruction_error(region_bounds=monitoring_regions, window_size=50)

🔬 Running simulation for 2000.0 time units (2000000 steps)...
✅ Simulation complete. Time taken: 13.55 seconds.

📊 Plotting standard results...
Saved: plot_1_position_vs_time.png
Saved: plot_2_velocity_vs_time.png
Saved: plot_3_bias_energy_vs_time.png
Saved: plot_4_final_energy_landscape.png

📈 Plotting evolving landscape snapshot...
Saved: plot_5_evolving_landscape.png

✨ Plotting point-wise force convergence (Window=50)...
Saved: plot_6_force_convergence_rolling_avg.png

⚡ Plotting Maximum Absolute Force Convergence (Window=50)...
Saved: plot_7_max_force_convergence.png

📏 Plotting MAE of FES Reconstruction (Window=50)...
Saved: plot_8_fes_mae_convergence.png
